## 5.11 残差网络（ResNet）

### 5.11.1 残差块

In [25]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from livelossplot.tf_keras import PlotLossesCallback

In [26]:
class Residual(keras.layers.Layer):
    def __init__(self, num_channels, use_1x1conv=False, strides=1, *args, **kwargs):
        super(Residual, self).__init__(*args, **kwargs)
        self.conv1 = keras.layers.Conv2D(num_channels, kernel_size=3, padding='same', strides=strides)
        self.conv2 = keras.layers.Conv2D(num_channels, kernel_size=3, padding='same')
        
        if use_1x1conv:
           self.conv3 = keras.layers.Conv2D(num_channels, kernel_size=1, strides=strides)
        else:
           self.conv3 = None
            
        self.bn1 = keras.layers.BatchNormalization()
        self.bn2 = keras.layers.BatchNormalization()
        
    def call(self, inputs, training=None, mask=None):
        Y = keras.layers.ReLU()(self.bn1(self.conv1(inputs)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
           inputs = self.conv3(inputs)
        return keras.layers.ReLU()(Y + inputs)
    

In [27]:
inp = keras.Input((4, 3, 3))
blk = Residual(3)(inp)
model = keras.Model(inp, blk)
model.output_shape

### 5.11.2 ResNet模型

In [28]:
model = keras.Sequential()
model.add(keras.layers.Lambda(lambda img: tf.image.resize(img, (224, 224)), input_shape=(28, 28, 1)))
model.add(keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same', input_shape=(224, 224, 1)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.ReLU())
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))

In [29]:
def resnet_block(model, num_channels, num_residuals, first_block=False):
    for i in range(num_residuals):
        if i == 0 and not first_block:
            model.add(Residual(num_channels, use_1x1conv=True, strides=2))
        else:
            model.add(Residual(num_channels))
    return model

In [30]:
resnet_block(model, 64, 2, first_block=True),
resnet_block(model, 128, 2),
resnet_block(model, 256, 2),
resnet_block(model, 512, 2)

In [31]:
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dense(10))
model.add(keras.layers.Softmax())

model.summary()

### 5.11.3 获取数据和训练模型

In [32]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train = x_train / 255.

In [ ]:
def metric_accuracy(y_true, y_pred):
    ytrue = K.flatten(y_true)
    ypred = K.cast(K.argmax(y_pred, axis=-1), K.floatx())
    acc = K.equal(ytrue, ypred)
    return K.mean(acc)

model.compile(optimizer=keras.optimizers.Adam(), 
              loss=keras.losses.sparse_categorical_crossentropy,
              metrics=[metric_accuracy])

model.fit(
    x_train.reshape((x_train.shape[0], 28, 28, 1)), 
    y_train, epochs=5, batch_size=128,
    #validation_data=(x_test.reshape(x_test.shape[0], 28, 28, 1), y_test),
    validation_split=0.3,
    callbacks=[PlotLossesCallback()])